In [1]:
import os
import datetime
from typing import Optional, Tuple

import gym
import numpy as np
import torch
from tianshou.data import Collector, VectorReplayBuffer, PrioritizedVectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager, RandomPolicy
from tianshou.trainer import offpolicy_trainer
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger

from TaskAllocation.RL_Policies.Custom_Classes import CustomNet
from TaskAllocation.RL_Policies.Custom_Classes import CustomCollector
from TaskAllocation.RL_Policies.Custom_Classes import CustomParallelToAECWrapper

#from CustomClass_multi_head import CustomNet
from TaskAllocation.RL_Policies.Custom_Classes_simplified import CustomNetSimple
#from Custom_Classes_simplified import CustomCollectorSimple
#from Custom_Classes_simplified import CustomParallelToAECWrapperSimple

from TaskAllocation.RL_Policies.CustomClasses_Transformer_Reduced import CustomNetReduced
from TaskAllocation.RL_Policies.CustomClass_MultiHead_Transformer import CustomNetMultiHead
import importlib

from mUAV_TA.DroneEnv import MultiDroneEnv
#from tianshou_DQN import train
model = "CustomNetMultiHead" # "CustomNet" or "CustomNetSimple" or "CustomNetReduced" or "CustomNetMultiHead"
test_num = "_Eval_TBTA_02_simplified_UCF_mask01_seed0All"

train_env_num = 10
test_env_num = 10

name = model + test_num

load_policy_name = f'policy_CustomNetMultiHead_Eval_TBTA_02_simplified_UCF1_new_rew_updR.pth'
save_policy_name = f'policy_{name}.pth'
policy_path = "dqn_Custom"
load_model = False

# log
now = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
log_name = name + str(now)

log_path = os.path.join('./', "Logs", "dqn", log_name)

dqn_params = {"discount_factor": 0.98, 
              "estimation_step": 5, 
              "target_update_freq": 100,
              "optminizer": "Adam",
              "lr": 1e-4  }

trainer_params = {"max_epoch": 1000,
                  "step_per_epoch": 250 * train_env_num,
                  "step_per_collect": 20,
                  "episode_per_test": 1,
                  "batch_size" : 32,
                  "update_per_step": 1/25,
                  "tn_eps_max": 0.85,
                  "ts_eps_max": 0.0,
                  }

Run_Data = f'{name}\n\
        Loaded_Model: {load_policy_name if load_model == True else "no"} \n\
        log_path: {log_path} \n\
        train/test_env_num: {train_env_num} / {test_env_num} \n\
        model: {model} \n\
        dqn_params: {dqn_params} \n\
        trainer_params: {trainer_params} \n\
        obs: Task Info -> Dist / Quality for own drone \
            Agents_info -> Post_next / Time_next / Type \
            Scene:  agents= F1:2, F2:2, R1:3, R2:3,     \
                 tasks= Att: 8 , Rec:22'

model_load_path = os.path.join(policy_path, load_policy_name)  
model_save_path = os.path.join(policy_path, save_policy_name)        
os.makedirs(os.path.join(policy_path), exist_ok=True)  
os.makedirs(os.path.join(log_path), exist_ok=True)


def _get_agents(
    agent_learn: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
    policy_load_path = None,
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    
    env = _get_env()
    agent_name = env.agents[0]  # Get the name of the first agent

    #print(env.observation_space )
    agent_observation_space = env.observation_space # assuming 'agent0' is a valid agent name
    state_shape_agent_position = agent_observation_space["agent_position"].shape[0]
    state_shape_agent_state = agent_observation_space["agent_state"].shape[0]
    state_shape_agent_type = agent_observation_space["agent_type"].shape[0]
    state_shape_next_free_time = agent_observation_space["next_free_time"].shape[0]
    state_shape_position_after_last_task = agent_observation_space["position_after_last_task"].shape[0]       
    #state_shape_agent_relay_area = agent_observation_space["agent_relay_area"].shape[0]
        
    state_shape_agent = (state_shape_agent_position + state_shape_agent_state +
                     state_shape_agent_type+ state_shape_next_free_time + state_shape_position_after_last_task #+                     
                     #state_shape_agent_relay_area
                     )                 
    

    state_shape_task = 30 * 3 #env.observation_space["tasks_info"].shape[0]
                  
    action_shape = env.action_space[agent_name].shape[0]
    #action_shape = env.action_space[agent_name].n
               
    if agent_learn is None:
        # model
        if model == "CustomNet":        
            net = CustomNet(
                state_shape_agent=state_shape_agent,
                state_shape_task=state_shape_task,
                action_shape=action_shape,
                hidden_sizes=[128,128],
                device="cuda" if torch.cuda.is_available() else "cpu",
            ).to("cuda" if torch.cuda.is_available() else "cpu")
        
        if model == "CustomNetSimple":
            net = CustomNetSimple(            
                state_shape_agent=state_shape_agent,
                state_shape_task=state_shape_task,
                action_shape=action_shape,
                hidden_sizes=[128,128],
                device="cuda" if torch.cuda.is_available() else "cpu",
            ).to("cuda" if torch.cuda.is_available() else "cpu")
        
        if model == "CustomNetReduced":
            net = CustomNetReduced(            
                state_shape_agent=state_shape_agent,
                state_shape_task=state_shape_task,
                action_shape=action_shape,
                hidden_sizes=[128,128],
                device="cuda" if torch.cuda.is_available() else "cpu",
            ).to("cuda" if torch.cuda.is_available() else "cpu")
        
        if model == "CustomNetMultiHead":
            net = CustomNetMultiHead(
                state_shape_agent=state_shape_agent,
                state_shape_task=state_shape_task,
                action_shape=action_shape,
                hidden_sizes=[128,128],
                device="cuda" if torch.cuda.is_available() else "cpu",
            ).to("cuda" if torch.cuda.is_available() else "cpu")

    
        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=dqn_params["lr"])
    
        agent_learn = DQNPolicy(
            model=net,
            optim=optim,
            discount_factor= dqn_params["discount_factor"],
            estimation_step=dqn_params["estimation_step"],
            target_update_freq=dqn_params["target_update_freq"],
        )  
        
        if load_model == True:
            # Load the saved checkpoint             
            agent_learn.load_state_dict(torch.load(model_load_path))
            print(f'Loaded-> {model_load_path}')
            
        
        agents = [agent_learn for _ in range(len(env.agents))]
        
    policy = MultiAgentPolicyManager(agents, env)    
        
    return policy, optim, env.agents


def _get_env():
    """This function is needed to provide callables for DummyVectorEnv."""   
    env_paralell = MultiDroneEnv()
    #env = parallel_to_aec_wrapper(env_paralell)    
    env = CustomParallelToAECWrapper(env_paralell)
    
    return PettingZooEnv(env)

print(Run_Data)

CustomNetMultiHead_Eval_TBTA_02_simplified_UCF_mask01_seed0All
        Loaded_Model: no 
        log_path: ./Logs\dqn\CustomNetMultiHead_Eval_TBTA_02_simplified_UCF_mask01_seed0All230915-112008 
        train/test_env_num: 10 / 10 
        model: CustomNetMultiHead 
        dqn_params: {'discount_factor': 0.98, 'estimation_step': 5, 'target_update_freq': 100, 'optminizer': 'Adam', 'lr': 0.0001} 
        trainer_params: {'max_epoch': 1000, 'step_per_epoch': 2500, 'step_per_collect': 20, 'episode_per_test': 1, 'batch_size': 32, 'update_per_step': 0.04, 'tn_eps_max': 0.85, 'ts_eps_max': 0.0} 
        obs: Task Info -> Dist / Quality for own drone             Agents_info -> Post_next / Time_next / Type             Scene:  agents= F1:2, F2:2, R1:3, R2:3,                      tasks= Att: 8 , Rec:22


In [2]:
if __name__ == "__main__":
                        
    torch.set_grad_enabled(True) 
    # ======== Step 1: Environment setup =========
    train_envs = DummyVectorEnv([_get_env for _ in range(train_env_num)])
    test_envs = DummyVectorEnv([_get_env for _ in range(test_env_num)]) 

    # seed
    seed = 1
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    train_envs.seed(seed)
    test_envs.seed(seed)

    # ======== Step 2: Agent setup =========
    policy, optim, agents = _get_agents()
    

    # ======== Step 3: Collector setup =========
    train_collector = CustomCollector(
        policy,
        train_envs,
        #VectorReplayBuffer(100_000, len(train_envs)),
        PrioritizedVectorReplayBuffer( 100_000, len(train_envs), alpha=0.6, beta=0.4) ,       
        exploration_noise=True             
    )
    test_collector = CustomCollector(policy, test_envs, exploration_noise=True)
     
    #train_collector.collect(n_step=trainer_params['batch_size'] * train_env_num)
    train_collector.collect(n_episode=10)
    #test_collector.collect(n_episode=2 )
    #test_collector.collect(n_step=trainer_params['batch size'] * train_env_num)
    
    # ======== tensorboard logging setup =========
    #         
    writer = SummaryWriter(log_path)
    writer.add_text(name, str(Run_Data))
    logger = TensorboardLogger(writer)
        
    # ======== Step 4: Callback functions setup =========
    def save_best_fn(policy):                
        print("Best Saved")
        torch.save(policy.policies[agents[0]].state_dict(), model_save_path)

    def stop_fn(mean_rewards):
        return mean_rewards >= 9939.0

    def train_fn(epoch, env_step):
        epsilon = trainer_params['tn_eps_max'] - (trainer_params['tn_eps_max'] - trainer_params['tn_eps_max']/100)*(epoch/trainer_params['max_epoch'])  
        policy.policies[agents[0]].set_eps(epsilon)

    def test_fn(epoch, env_step):
        epsilon = trainer_params['ts_eps_max']#0.01#max(0.001, 0.1 - epoch * 0.001)
        policy.policies[agents[0]].set_eps(epsilon)
        
    def reward_metric(rews):       
        #print(rews)
        return rews.mean()#[:,0]
                           
    # ======== Step 5: Run the trainer =========
    result = offpolicy_trainer(
        policy=policy,
        train_collector=train_collector,
        test_collector=test_collector,        
        max_epoch=trainer_params['max_epoch'],
        step_per_epoch=trainer_params['step_per_epoch'],
        step_per_collect=trainer_params['step_per_collect'],
        episode_per_test= trainer_params['episode_per_test'],
        batch_size=trainer_params['batch_size'],
        train_fn=train_fn,
        test_fn=test_fn,
        stop_fn=stop_fn,
        save_best_fn=save_best_fn,
        update_per_step=trainer_params['update_per_step'],
        logger=logger,
        test_in_train=False,
        reward_metric=reward_metric,
        show_progress = True          
        )

    # return result, policy.policies[agents[1]]
    print(f"\n==========Result==========\n{result}")
    print("\n(the trained policy can be accessed via policy.policies[agents[0]])")

d:\Python310\lib\site-packages\torch\nn\init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Best Saved


Epoch #1: 2501it [00:25, 97.85it/s, agent0/loss=417.506, agent1/loss=373.098, agent2/loss=471.721, agent3/loss=331.841, agent4/loss=258.785, agent5/loss=285.056, agent6/loss=271.582, agent7/loss=228.886, agent8/loss=216.837, agent9/loss=291.650, env_step=2500, len=50, n/ep=10, n/st=20, rew=124.65]                           


Best Saved
Epoch #1: test_reward: 185.000000 ± 0.000000, best_reward: 185.000000 ± 0.000000 in #1


Epoch #2: 2501it [00:26, 94.17it/s, agent0/loss=375.417, agent1/loss=303.714, agent2/loss=387.532, agent3/loss=322.924, agent4/loss=233.661, agent5/loss=251.161, agent6/loss=211.441, agent7/loss=156.173, agent8/loss=183.473, agent9/loss=187.043, env_step=5000, len=50, n/ep=10, n/st=20, rew=130.45]                           


...

Epoch #47: test_reward: 185.000000 ± 0.000000, best_reward: 185.000000 ± 0.000000 in #1


Epoch #48: 2501it [00:24, 101.11it/s, agent0/loss=318.454, agent1/loss=300.936, agent2/loss=306.291, agent3/loss=283.871, agent4/loss=260.479, agent5/loss=216.311, agent6/loss=231.276, agent7/loss=174.063, agent8/loss=184.184, agent9/loss=229.167, env_step=120000, len=50, n/ep=10, n/st=20, rew=130.05]                          


Epoch #48: test_reward: 185.000000 ± 0.000000, best_reward: 185.000000 ± 0.000000 in #1


Epoch #49:  38%|###7      | 940/2500 [00:10<00:17, 91.67it/s, agent0/loss=305.855, agent1/loss=270.213, agent2/loss=296.074, agent3/loss=266.801, agent4/loss=272.704, agent5/loss=227.276, agent6/loss=199.069, agent7/loss=187.088, agent9/loss=207.807, env_step=120920, len=50, n/ep=0, n/st=20, rew=137.35]                       


KeyboardInterrupt: 

In [ ]:
from typing import Optional, Tuple
import os
import numpy as np
import torch
from tianshou.env import DummyVectorEnv
from tianshou.trainer import offpolicy_trainer
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger
import torch

import mUAV_TA.MultiDroneEnvUtils as utils
#from Custom_Classes import CustomCollector

def _get_env_eval():
    """This function is needed to provide callables for DummyVectorEnv."""
    case =  {'case' : 0, 'F1':2, 'F2': 2, "R1" : 3, 'R2' : 3, "Att" : 8, "Rec" : 22}

    config = utils.DroneEnvOptions(     
            render_mode = 'human', 
            render_speed = -1,
            simulation_frame_rate = 1 / 60,
            simulator_module = "Internal", 
            max_time_steps = 300,
            action_mode= "TaskAssign",
            agents= {"F1" : case['F1'], "F2" : case['F2'], "R1" : case['R1'], "R2" : case['R2']},                 
            tasks= { "Att" : case['Att'], "Rec" : case['Rec']},
            random_init_pos = True,
            num_obstacles = 0,
            hidden_obstacles = False,
            fail_rate = 0,
            info = "No Info" )
   
    
    env_paralell = MultiDroneEnv()
    #env = parallel_to_aec_wrapper(env_paralell)    
    env = CustomParallelToAECWrapper(env_paralell)
    
    return PettingZooEnv(env)


# Create a new instance of the policy with the same architecture as the saved policy
name = 'policy_CustomNetMultiHead_Eval_TBTA_02_simplified_UCF_mask01_seed0All.pth' 
load_policy_name = f'policy_{name}'


log_path = os.path.join('./', "Logs", "dqn", name)

policy, optim, _ = _get_agents()
model_save_path = os.path.join("dqn_Custom", save_policy_name)        

# Load the saved checkpoint
policy_test = policy.policies['agent0']
policy_test.load_state_dict(torch.load(model_save_path ))

envs = DummyVectorEnv([_get_env_eval for _ in range(1)])
policy_test.eval()
policy_test.set_eps(0.00)

#collector = CustomCollector(policy.policies['agent0'], envs, exploration_noise=True)
#collector = CustomCollector(policy_test, envs, exploration_noise=False)
collector = CustomCollector(policy, envs, exploration_noise=True)

#results = collector.collect(n_episode=1)
results = collector.collect(n_episode=5)#, gym_reset_kwargs={'seed' :2})
results

In [ ]:
results['rews']
print(np.mean(results['rews'][results['rews'] > -10]))


#create a function  to print a histogram of the results['rews']
import matplotlib.pyplot as plt
plt.hist(results['rews'][results['rews'] > -10], bins=100)
plt.show()


In [ ]:
from turtle import st
import torch
from tianshou.data import Batch

# load policy as in your original code
policy, optim, _ = _get_agents()
model_save_path = os.path.join("dqn_Custom", save_policy_name)        
policy_test = policy.policies['agent0']
state_saved = torch.load(model_save_path )
#print(policy_test)
policy_test.load_state_dict(state_saved)
policy_test.eval()
policy_test.set_eps(0.00)

# initialize your environment
#env = DummyVectorEnv([_get_env for _ in range(1)])
env = MultiDroneEnv(None)
env.max_time_steps = 50

# simulate the interaction with the environment manually
for i in range(10):
    for episode in range(1):  # simulate 10 episodes
        
        #env.render_speed = 1
        obs, _  = env.reset(seed=episode)         
        info         = env.get_initial_state()
        
        drones = info["drones"]
        tasks = info["tasks"]
            
        done = {0 : False}
        truncations = {0 : False}
        
        episodo_reward = 0
        #obs, reward, done, truncations, info = env.step(action)

        while not all(done.values()) and not all(truncations.values()):
            
            agent_id = "agent" + str(env.agent_selector._current_agent)
            # Create a Batch of observations
            obs_batch = Batch(obs=[obs[agent_id]], info=[{}])  # add empty info for each observation
            
            #print(obs_batch)
            # Forward the batch of observations through the policy to get the actions
            action = policy_test(obs_batch).act
            action = {agent_id : action[0]}
        
            obs, reward, done, truncations, info = env.step(action)
            
            episodo_reward += sum(reward.values())/env.n_agents

        

    print(episodo_reward)
